In [7]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForQuestionAnswering
from datasets import Dataset, load_dataset

import nltk
from nltk.translate.bleu_score import corpus_bleu

from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

In [8]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

# Load dataset directly
dataset = load_dataset("vibhorag101/phr_mental_therapy_dataset")

In [9]:
dataset['train']['text'][:2]

["<s>[INST] <<SYS>>\nYou are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>\n\nI've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me. [/INST] Hey there, I'm here to listen and support you. It sounds like work has been really challenging lately. Can you tell me more about what's been going on? </s><s>[INST] I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. I

In [44]:
import pandas as pd

# Initialize lists to store inputs and responses
inputs = []
responses = []

# Iterate over each conversation
for conversation in dataset['train']['text']:
    # Remove "<s>[INST] <<SYS>>\n" prefix
    #conversation = conversation.replace("<s>[INST] <<SYS>>\n", "")
    
    # Split the conversation into individual lines after "</INST>"
    lines = conversation.split('INST]')
    
    # Extract inputs (odd-indexed lines) and responses (even-indexed lines)
    for i in range(len(lines) - 1):  # Adjusted range to avoid IndexError
        if i % 2 == 0:  # Odd-indexed lines are inputs
            responses.append(lines[i].strip())
        else:  # Even-indexed lines are responses
            inputs.append(lines[i].strip())

# Create a DataFrame
data = pd.DataFrame({'input': inputs, 'response': responses})

data = data[data['response']!="<s>["]

In [43]:
data

,input,response
1,"I recently got a promotion at work, which I th...","Hey there, I'm here to listen and support you...."
2,"Well, the workload has increased significantly...",I can understand how it can be overwhelming wh...
3,I've been trying to prioritize my tasks and de...,It sounds like you're dealing with a lot of pr...
4,You're right. I haven't really opened up about...,It's great to hear that you're already impleme...
5,Thank you for your understanding and guidance....,"It's completely normal to feel that way, but r..."
7,"Well, lately, I've been feeling like my friend...",Hello there! I'm here to listen and offer supp...
8,"No, I haven't. I'm afraid that they'll think I...",I understand how disheartening that can be. It...
9,"You're right, I should talk to them. But how d...",It's normal to feel hesitant about having diff...
10,"I see, so it's important to frame it as a dial...",That's a great question! It can be helpful to ...
11,It's reassuring to hear that. I'll try to gath...,Absolutely! Opening up a conversation can lead...


In [46]:
# Split the dataset into train, validation, and test sets
train_inputs, val_test_inputs, train_responses, val_test_responses = train_test_split(data['input'], data['response'], test_size=0.2, random_state=42)
val_inputs, test_inputs, val_responses, test_responses = train_test_split(val_test_inputs, val_test_responses, test_size=0.5, random_state=42)

In [49]:
#  Combine inputs and responses into a dataset for training
train_dataset = Dataset.from_dict({'input': train_inputs, 'response': train_responses})
val_dataset = Dataset.from_dict({'input': val_inputs, 'response': val_responses})

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    num_train_epochs=3,
    logging_steps=100,
    output_dir="./output"
)

# Load a pre-trained BLEU model
def compute_bleu_score(preds, targets):
    # Convert predictions and targets to lists of strings
    preds = [str(pred) for pred in preds]
    targets = [[str(target)] for target in targets]  # Convert targets to list of lists for corpus_bleu function
    bleu_score = corpus_bleu(targets, preds)
    return bleu_score

# Load a pre-trained STS model
model2 = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def compute_sts(preds, targets):
    # Encode predictions and targets
    pred_embeddings = model2.encode(preds)
    target_embeddings = model2.encode(targets)

    # Compute cosine similarity between embeddings
    sts_scores = [1 - cosine(pred_embedding, target_embedding) for pred_embedding, target_embedding in zip(pred_embeddings, target_embeddings)]
    return sts_scores

# Define evaluation function
def compute_metrics(eval_pred):
    preds, labels = eval_pred.predictions, eval_pred.label_ids
    bleu_score = compute_bleu_score(preds, labels)
    sts_score = compute_sts(preds, labels)
    return {"bleu_score": bleu_score, "sts_score": sts_score}

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune the model
trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`